## Import Model

In [1]:
from transformers import AutoModel, AutoTokenizer, set_seed
import torch

# Set random seed for reproducibility
set_seed(42)

# Model and tokenizer
model_name = "elgeish/gpt2-medium-arabic-poetry"
model = AutoModel.from_pretrained(model_name).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Input prompt
prompt = "مثلجات سلطع برجر"
input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")

# Ensure the model is in evaluation mode
model.eval()

# Get the model outputs
with torch.no_grad():
    outputs = model(input_ids)

# Extract hidden states from the last layer
hidden_states = outputs.last_hidden_state

# Calculate the sentence embedding as the mean of the token embeddings
sentence_embedding = torch.mean(hidden_states, dim=1).squeeze()

# Convert to numpy array (optional)
sentence_embedding = sentence_embedding.cpu().numpy()

print("Sentence embedding:", sentence_embedding)

2024-06-08 23:58:23.874323: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-08 23:58:24.512025: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-08 23:58:27.171987: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Sentence embedding: [-1.7680897  -0.9779109   0.34898052 ... -0.3351004  -0.95397943
 -0.3652193 ]


In [2]:
import pandas as pd

df = pd.read_csv('data/apcd_clean.csv')
df.head()

,البيت
0,خليلي لا تستعجلا أن تزودا وأن تجمعا شملي وت...
1,فما لبث يوما بسابق مغنم ولا سرعتي يوما بساب...
2,وإن تنظراني اليوم أقض لبانة وتستوجبا منا عل...
3,لعمرك ما نفس بجد رشيدة تؤامرني سرا لأصرم مرثدا
4,وإن ظهرت منه قوارص جمة وأفرع في لومي مرارا ...


In [3]:
from torch.utils.data import DataLoader, Dataset

set_seed(42)

class PromptDataset(Dataset):
    def __init__(self, prompts):
        self.prompts = prompts

    def __len__(self):
        return len(self.prompts)

    def __getitem__(self, idx):
        prompt = self.prompts[idx]
        input_ids = tokenizer.encode(prompt, return_tensors="pt").squeeze(0)
        return input_ids

# Create Dataset and DataLoader
dataset = PromptDataset(df.sample(10_000, random_state=42)['البيت'].values)
dataloader = DataLoader(dataset, batch_size=64, collate_fn=lambda x: torch.nn.utils.rnn.pad_sequence(x, batch_first=True))

# Ensure the model is in evaluation mode
model.eval()

# Function to extract embeddings
def extract_embeddings(dataloader, model):
    all_embeddings = []
    with torch.no_grad():
        for batch in dataloader:
            batch = batch.to("cuda")
            outputs = model(batch)
            hidden_states = outputs.last_hidden_state
            # Calculate the sentence embedding as the mean of the token embeddings
            batch_embeddings = torch.mean(hidden_states, dim=1)
            all_embeddings.append(batch_embeddings)
            
            del batch
            del outputs
            del hidden_states
            torch.cuda.empty_cache()
    # Concatenate all embeddings
    all_embeddings = torch.cat(all_embeddings, dim=0)
    return all_embeddings

# Extract embeddings
embeddings = extract_embeddings(dataloader, model)

# Convert to numpy array (optional)
embeddings = embeddings.cpu().numpy()

print("Embeddings shape:", embeddings.shape)
print(embeddings)

Embeddings shape: (10000, 1024)
[[-0.13975072 -0.54131925  0.89815    ...  1.9655814   1.9515972
   0.01922477]
 [-0.09095679 -0.84680843 -0.64323944 ...  0.00599787  1.9318521
   0.5713279 ]
 [-0.64327407 -1.5250567  -0.3005728  ...  1.0522782   1.817242
   0.7140913 ]
 ...
 [-0.14110458 -1.3170115   0.7179195  ... -0.03673433 -0.19882926
   0.24241851]
 [-0.02675416 -0.19866264  0.73517567 ...  0.26320434  1.4981219
  -1.0008411 ]
 [ 0.23260784 -0.22998694  0.12302577 ...  0.85278064  1.686227
  -0.3900748 ]]


In [4]:
import faiss

dimension = embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
print(f'Total vectors in the index: {index.ntotal}')

Total vectors in the index: 10000


In [5]:
original_indices = df.sample(10_000, random_state=42)['البيت'].index

In [7]:
prompts = [
    'الحماسة والفخر',
    'روعة البحر',
    'الأمل والتفاؤل',
    'معاني الحياة والموت',
    'وجدانية الحب العذري',
    'التأمل في مصائر الإنسان',
    'حياة العرب قبل الإسلام',
    'العدمية والبحث عن الذات',
    'الفضيلة والأخلاق',
    'السلام والتسامح',
    'العدالة والمساواة',
    'كيف تخاف الفقر والله رازقا',
    'التشرد والهجرة والبحث عن الوطن',
    'عيناك نازلتاالقلوب',
    ' كتب الدمع بخدي عهده',
    'لا يحمل الحقد من تعلو به الرتب',
    'أتعرف رسم الدار',
    'الأصدقاء مثل النجوم',
    ' أحلك الأوقات',
    'شعر بدوي',
    ' الثقة بالله',
    ' ذكريات الطفولة',
    'المرء يأمل أن يعيش',
    'الوطن والانتماء',
    ' الحرية والنضال',
    'قم للمعلم وفه التبجيلا',
    'وعذرتهم وعرفت ذنبي',
    'إليك يا أمي',
    'فأصبح بعد الحرب',
    'إذا حكم القضاء',
    'ليت الأحزان',
    'فرحي مثل طائر',
    'نار الغضب',
    'صوت المطر ينعش روحي',
    'تنير ليالي',
    'إِن الثعالب بالضحى',
    'صبور على مر الحوادث',
    'البحث عن الحقيقة رحلة لا نهاية لها',
    'مرآة الروح',
    'كلما نسينا دروس الماضي',
    'دع الأيام تفعل ما تشاء',
    'وقفت على الديار',
    'رمال الصحراء',
    'شموخ',
    'تفاؤل',
    'نصر',
    'حمامة',
    'ساذج'
]

f = open("output_gpt2.txt", 'w')

for prompt in prompts:
    f.write(prompt + "\n")

    set_seed(42)

    input_ids = tokenizer.encode(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model(input_ids)

    # Extract hidden states from the last layer
    hidden_states = outputs.last_hidden_state

    # Calculate the sentence embedding as the mean of the token embeddings
    sentence_embedding = torch.mean(hidden_states, dim=1).squeeze()

    # Convert to numpy array (optional)
    query_vector = sentence_embedding.cpu().numpy().reshape(1, -1)
    
    distances, indices = index.search(query_vector, 50)
    df_prompt_result = df.loc[original_indices[indices.flatten()]]
    df_prompt_result['n'] = df_prompt_result['البيت'].apply(len)
    df_prompt_result = df_prompt_result.loc[df_prompt_result['n'] > 10]
    df_prompt_result.reset_index(inplace=True)
    
    f.write(df_prompt_result.at[0, 'البيت'] + "\n")
    f.write(df_prompt_result.at[1, 'البيت'] + "\n")
    f.write(df_prompt_result.at[2, 'البيت'] + "\n")
    
    f.write("\n")

f.close()

In [5]:
#faiss.write_index(index, 'vector_index.faiss')